In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fpdf import FPDF
from matplotlib.ticker import FuncFormatter

import shutil
import subprocess
import sys

# Formata os números com separadores de milhar (ex: 1.000, 10.000)
formatter = FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.'))


sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [8]:
caminho = '../../dados_tratados/csv/'

dados = {
    'alunos': pd.read_csv(caminho+'escolas_alunos_2006.csv', sep=';', encoding='utf-8'),
    'professores': pd.read_csv(caminho+'escolas_professores_2006.csv', sep=';', encoding='utf-8'),
}

display(dados['alunos'].head())

,nome_escola,cidade_escola,local_bh_rmbh,bairro_escola,turma,hora_inicio,hora_fim,aplicadores,observacoes,data_aplicacao,...,participacao_programa,participacao_pais,pertence_gangue,envolvimento_gangues,ja_furtou,uso_bebida,uso_cigarro,uso_solventes,uso_maconha,uso_crack
0,e.e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia e nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,escola estadual celso machado,belo horizonte,bh,milionarios,2,36600,38700.0,adenir,NaN,13362192384,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0
2,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0
3,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery e klarissa,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,e. e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia/nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [9]:
print(dados['alunos']['raca_cor'].str.lower().value_counts().get('pardo', 0))

855


In [10]:
# Criar o diretório temp/ se não existir
temp_dir = 'temp'
os.makedirs(temp_dir, exist_ok=True)

# Selecionar um subconjunto de colunas e renomeá-las para facilitar a manipulação
colunas_selecionadas = {
    'nome_escola': 'nome_escola',
    'bairro': 'localizacao',
    'idade': 'idade',
    'sexo': 'sexo',
    'raca_cor': 'raca_cor',
    'satisfacao_aprendizado': 'satisfacao_aprendizado',
    'ocorrencia_roubo': 'ocorrencia_roubo',
    'ocorrencia_furto': 'ocorrencia_furto',
    'ocorrencia_agressao': 'ocorrencia_agressao'
}
df_analise = dados['alunos'][list(colunas_selecionadas.keys())].rename(columns=colunas_selecionadas)

# Limpeza de dados
# Converter 'idade' para numérico e remover valores não válidos
df_analise['idade'] = pd.to_numeric(df_analise['idade'], errors='coerce')
df_analise.dropna(subset=['idade'], inplace=True)
df_analise['idade'] = df_analise['idade'].astype(int)

# Filtrar idades para um intervalo razoável (removendo outliers como 1300)
df_analise = df_analise[(df_analise['idade'] >= 5) & (df_analise['idade'] <= 20)]

# Remover valores 'na' e 'nr' das colunas categóricas
for col in ['sexo', 'raca_cor', 'satisfacao_aprendizado', 'localizacao']:
    df_analise = df_analise[~df_analise[col].astype(str).isin(['na', 'nr', 'nan'])]

# --- IMPLEMENTAÇÃO DAS VISUALIZAÇÕES SOLICITADAS ---

plt.style.use('seaborn-v0_8-whitegrid')

# Gráfico 1: Distribuição de Alunos por Idade
plt.figure(figsize=(12, 6))
contagem_idade = df_analise['idade'].value_counts().sort_index()
sns.barplot(x=contagem_idade.index, y=contagem_idade.values, palette='coolwarm')
plt.plot(contagem_idade.index, contagem_idade.values, color='black', marker='o', linestyle='-', linewidth=2)
plt.title('Distribuição de Alunos por Idade', fontsize=16)
plt.xlabel('Idade', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(temp_dir+'/distribuicao_idade.png')


df_analise = df_analise[df_analise['raca_cor'] != 'uma outra mistura']

# Gráfico 2 e 3: Distribuição de Alunos por Raça/Cor (Gráficos de Pizza)
raca_counts = df_analise['raca_cor'].value_counts()
top_n = 5 # Define o número de categorias principais a serem exibidas
top_races = raca_counts.head(top_n)
other_races = raca_counts.tail(-top_n)

# Gráfico de Pizza 1: Categorias Principais
if not top_races.empty:
    pie1_data = top_races.copy()
    if not other_races.empty:
        pie1_data['Outros'] = other_races.sum()

    plt.figure(figsize=(10, 7))
    plt.pie(pie1_data, labels=pie1_data.index, autopct='%1.1f%%', startangle=140,
            wedgeprops={'edgecolor': 'white'}, textprops={'fontsize': 12})
    plt.title('Distribuição Principal por Raça/Cor', fontsize=16)
    plt.axis('equal')
    plt.savefig(temp_dir+'/distribuicao_raca_principal.png')

# Gráfico de Pizza 2: Detalhamento de "Outros"
if not other_races.empty:
    plt.figure(figsize=(10, 7))
    plt.pie(other_races, labels=other_races.index, autopct='%1.1f%%', startangle=90,
            wedgeprops={'edgecolor': 'white'}, textprops={'fontsize': 12})
    plt.title('Detalhamento das Demais Categorias de Raça/Cor', fontsize=16)
    plt.axis('equal')
    plt.savefig(temp_dir+'/distribuicao_raca_outros.png')

# Demais gráficos para consistência da análise
# Gráfico 4: Distribuição por Sexo
plt.figure(figsize=(10, 6))
sns.countplot(x='sexo', data=df_analise, order=df_analise['sexo'].value_counts().index, palette='viridis')
plt.title('Distribuição de Alunos por Sexo', fontsize=16)
plt.xlabel('Sexo', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(temp_dir+'/distribuicao_sexo.png')

# Gráfico 5: Satisfação com o aprendizado
plt.figure(figsize=(10, 6))
sns.countplot(y='satisfacao_aprendizado', data=df_analise, order=df_analise['satisfacao_aprendizado'].value_counts().index, palette='plasma')
plt.title('Satisfação dos Alunos com o Aprendizado', fontsize=16)
plt.xlabel('Contagem', fontsize=12)
plt.ylabel('Nível de Satisfação', fontsize=12)
plt.tight_layout()
plt.savefig(temp_dir+'/satisfacao_aprendizado.png')

plt.close('all') # Fecha todas as figuras para não exibi-las no output final desnecessariamente

C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\2970636297.py:39: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=contagem_idade.index, y=contagem_idade.values, palette='coolwarm')
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\2970636297.py:82: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='sexo', data=df_analise, order=df_analise['sexo'].value_counts().index, palette='viridis')
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\2970636297.py:92: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y='satisfacao_aprendizado', data=df_analis

In [11]:
# --- Bloco 2: Célula para gerar o PDF e apagar a pasta temp/ ---
# Este é o código principal que você solicitou.

# Tenta instalar a biblioteca fpdf2, necessária para criar o PDF
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "fpdf2"])
    from fpdf import FPDF
except (subprocess.CalledProcessError, ImportError) as e:
    print(f"Erro: Não foi possível instalar ou importar a biblioteca fpdf2. {e}")
    print("Por favor, instale-a manualmente com 'pip install fpdf2' e tente novamente.")
    # Se não conseguir instalar, não continua a execução para evitar mais erros.
    sys.exit(1)


# Define a estrutura do PDF com os caminhos das imagens e as análises
pdf_content = [
    {
        "title": "Distribuição de Alunos por Idade",
        "image": "distribuicao_idade.png",
        "analysis": (
            "O gráfico de barras e linha sobreposto mostra a contagem de alunos por idade. "
            "A análise da distribuição etária revela a concentração dos estudantes em determinadas faixas, "
            "possibilitando a identificação de picos de matrículas e a adequação de políticas para cada grupo."
        )
    },
    {
        "title": "Distribuição de Alunos por Raça/Cor",
        "image": "distribuicao_raca_principal.png",
        "analysis": (
            "O primeiro gráfico de pizza exibe a proporção das principais categorias de raça/cor declaradas, "
            "com uma categoria 'Outros' que agrupa as demais. Oferece uma visão geral da diversidade racial."
        )
    },
    {
        "title": "Detalhamento das Demais Categorias de Raça/Cor",
        "image": "distribuicao_raca_outros.png",
        "analysis": (
            "O segundo gráfico de pizza detalha a composição da categoria 'Outros', "
            "permitindo uma análise mais aprofundada da diversidade, incluindo grupos minoritários."
        )
    },
    {
        "title": "Distribuição de Alunos por Sexo",
        "image": "distribuicao_sexo.png",
        "analysis": (
            "Este gráfico de barras ilustra a distribuição de alunos por sexo. A análise da proporção "
            "é fundamental para entender a dinâmica de gênero no ambiente escolar e promover a equidade."
        )
    },
    {
        "title": "Satisfação dos Alunos com o Aprendizado",
        "image": "satisfacao_aprendizado.png",
        "analysis": (
            "O gráfico de barras horizontais apresenta os níveis de satisfação dos alunos. "
            "A avaliação da satisfação discente é um indicador crucial da qualidade do ensino, "
            "sinalizando pontos fortes e áreas que necessitam de melhoria."
        )
    }
]

# Classe para criar o PDF com cabeçalho e rodapé
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Análise Gráfica de Dados de Alunos', 0, 1, 'C')
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, text):
        self.set_font('Arial', '', 12)
        # Usamos encode/decode para evitar problemas com caracteres especiais
        self.multi_cell(0, 10, text.encode('latin-1', 'replace').decode('latin-1'))
        self.ln()

    def add_chapter(self, title, image_path, analysis_text):
        self.add_page()
        self.chapter_title(title)
        # Adiciona a imagem, centralizada
        if os.path.exists(image_path):
            self.image(image_path, x=30, w=150) # x=posição, w=largura
            self.ln(5)
        else:
            self.set_font('Arial', 'I', 10)
            self.cell(0, 10, f"[Imagem não encontrada em '{image_path}']", 0, 1)
        self.chapter_body(analysis_text)


# Gerar o PDF
pdf = PDF()
pdf_filename = '../../analises/relatorio_alunos_escolas.pdf'

for item in pdf_content:
    image_full_path = os.path.join(temp_dir, item['image'])
    pdf.add_chapter(item['title'], image_full_path, item['analysis'])

pdf.output(pdf_filename)
print(f"PDF '{pdf_filename}' gerado com sucesso.")

# Apagar a pasta temp/ e seu conteúdo
try:
    shutil.rmtree(temp_dir)
    print(f"Pasta temporária '{temp_dir}/' foi apagada com sucesso.")
except OSError as e:
    print(f"Erro ao apagar a pasta '{temp_dir}': {e.strerror}")


C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\1619432028.py:64: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\1619432028.py:65: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Análise Gráfica de Dados de Alunos', 0, 1, 'C')
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\1619432028.py:74: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\1619432028.py:75: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title, 0, 1, 'L')
C:\Users\pedro\AppData\Local\Temp\ipykernel_2992\16194320

PDF '../../analises/relatorio_alunos_escolas.pdf' gerado com sucesso.
Pasta temporária 'temp/' foi apagada com sucesso.
